<a href="https://colab.research.google.com/github/kyuenjpl/ARSET_XCO2/blob/main/OCO2_2018_csv_using_basemap_16_day_avg_using_OPENCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to create frames by 16-day average for EACH month

In [ ]:
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os 
# for GIF creation
import PIL as plw 

# Animation and Graphics libraries
import plotly.express as px
import plotly
import seaborn as sns; sns.set()

### The code below will generate FRAMES of 16 day cycle for each months:
* Example: For January: 1-16 and 17-31, same goes for every other months
* the frames will be generated using library "BASEMAP"

In [ ]:
# create a function that creates a BASEMAP
def map_by_month(data, year, month, file_num,  x="Longitude",y= "Latitude", lon_min= -180, lon_max= 180, lat_min= -90, lat_max= 90, size_point= 5, frontier= False):
     
    # EG: getting the MAP region, here below coordinates gets the map for INDIA region only             
    # x="Longitude",y= "Latitude", lon_min= 60, lon_max= 100, lat_min= 0, lat_max= 50, size_point= 5, frontier= False):
    
    # Figure setting    
    plt.figure(figsize= (40, 22), edgecolor='b')
    
    #PROJECTION 'cyl' -> Equilateral Diagram
    m= Basemap(projection='cyl', resolution='l',
               llcrnrlat= lat_min, urcrnrlat=lat_max, llcrnrlon= lon_min, urcrnrlon= lon_max)

    m.scatter( data['Longitude'], data['Latitude'], c= data['Xco2'], cmap= plt.cm.turbo, s=30, #norm= normal,
             marker='d')
    
    # Customizing the COLORBAR
    cbar= plt.colorbar(orientation= "horizontal",
                      format= "%.0f",
                      extend="both",
                      shrink= 0.45,
                      aspect= 10,
                      pad= 0.04)
    
    cbar.set_label(label="XCO2 (parts per million)", size= 40)
    cbar.ax.tick_params(labelsize= 35)

    # ADJUSTT the CBAR by year
    cbar.set_ticks(np.linspace(390, 420, 5))
    plt.clim(390, 420)
    
    # BORDER Lining
    m.drawmapboundary(fill_color='black')
    
    # changing BACKGROUND color
    m.fillcontinents(color='grey', alpha= 0.25)
    plt.title('Orbiting Carbon Observatory- 2\nAtmospheric Carbon Dioxide Concentration\n'+str(year)+'-'+str(month), fontsize =50)
    
    # Saving the FRAMES into specified Directory
    # Change the PATH for desired location
    
    path ="frames" +'/'
    plt.savefig(path+str(file_num)+"_"+ str(month)+"_"+".jpeg",
                dpi= 50)


# DASK: To read multiple files(CSV)
- NOTE: Multiple files should be on the same directory

In [ ]:
from dask import dataframe as dd

In [ ]:
# READ FILES using DASK dataframe
dask_2018= dd.read_csv('/2018_csv/*.txt',
                     parse_dates=['DateTime'])

In [ ]:
dask_2018

,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day
npartitions=350,,,,,,,,
,float64,float64,float64,int64,datetime64[ns],int64,int64,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


## Conv: dask dataframe to PANDAS
- NOTE: this takes longer depending on the processor you are running

In [ ]:
%%time
df_2018= dask_2018.compute()
len(df_2018)

Wall time: 2min 31s


34611950

In [ ]:
df_2018.head(3)

,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day
0,399.74167,-73.16891,-152.27951,0,2018-01-01 00:59:28.370,2018,1,1
1,400.76410,-73.15197,-152.30045,0,2018-01-01 00:59:28.770,2018,1,1
2,399.90845,-73.15309,-152.36101,0,2018-01-01 00:59:28.780,2018,1,1


## Data cleaning:
- Reducing the size of the files by rounding decimals

In [ ]:
df_2018['Latitude']= df_2018['Latitude'].round(3)
df_2018['Longitude']= df_2018['Longitude'].round(3)
df_2018['Xco2']= df_2018['Xco2'].round(3)
df_2018.head(3)

,Xco2,Latitude,Longitude,quality_flag,DateTime,Year,Month,Day
0,399.742,-73.169,-152.280,0,2018-01-01 00:59:28.370,2018,1,1
1,400.764,-73.152,-152.300,0,2018-01-01 00:59:28.770,2018,1,1
2,399.908,-73.153,-152.361,0,2018-01-01 00:59:28.780,2018,1,1


# Generate FRAMES:
- 1. Generate Frames on the specified directory (located inside basemap function)
- 2. Using Library openCV to generate animation from from frames

## PATH to file 

In [ ]:
#create a Folder 'frames' on the same directory to save frames

current_directory= os.getcwd()
frames_folder= os.path.join(current_directory, r'frames')

if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)

In [ ]:
def first_16_day_cycle(data, year): 

    # first 16 days
    first_half= np.arange(1, 17)
    #second 16 days
    second_half= np.arange(17, 32)
    
    # months
    month= ['January', 'February','March', 'April','May','June','July','August','September','October','November','December']

    # total files
    file_num = 0
    
    # repition of months
    rep =0
  
    months_count= 0
    for j in range(1, 13):


        # for different months 1-12
        vol_1= data[(data['Month']== j)]
                
        # for first 16 days
        first_vol= vol_1 [ vol_1['Day'].isin(first_half)]
        map_by_month(first_vol, str(year),month[rep], file_num)

        
        file_num +=1
        # Second Frame: 16 day cycle (17-31)   
        second_vol= vol_1 [ vol_1['Day'].isin(second_half)]
        map_by_month( second_vol, str(year),month[rep], file_num )                 

        
        file_num +=1
        # Third FRAME: full frame of the entire month
        map_by_month(vol_1, str(year), month[rep], file_num)


#        file num increments
        file_num +=1

        # next months
        rep +=1
 

# TESTING for one year

In [ ]:
%%time
first_16_day_cycle(df_2018, 2018)

# Creating a GIF file format using the FRAMES above created
* passing the PATH of frames created

## Locate the path to FRAMES
- Uses the frames created by basemap to generate an animation

In [ ]:
import cv2
import os
from os.path import isfile, join
import re
import natsort
from IPython.display import Video, HTML

In [ ]:
def two_convert_to_mp4_a(pathIn, pathOut, fps, time):
    
    frame_arr= []
    
    image_folder = 'images'
  #  video_name = 'video.avi'

    images = [img for img in os.listdir(pathIn) if img.endswith(".jpeg")]
    frame = cv2.imread(os.path.join(pathIn, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(pathOut, 0, 1, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(pathIn, image)))

    cv2.destroyAllWindows()
    video.release()


## Call the function to generate MP4 file
- Function below will generate an animation by combining the frames
- File name: oco2_anim_a

In [ ]:
pathIn= 'frames/'
pathOut= pathIn +'oco2_anim_a.mp4'
fps= 1
time= 1

two_convert_to_mp4_a(pathIn, pathOut, fps, time)

In [ ]:
Video('frames/oco2_anim_a.mp4', embed= True, width = 800, height= 500)